In [ ]:
#This is currently a work in progress
#more comments and data extraction to come!
#to properly render this notebook go to: 
#    http://nbviewer.ipython.org/github/BEugeneSmith7/codingFashion/blob/master/NordstromDescStats.ipynb

#go back and code each fit, pattern, item, and texture into one of these four

In [ ]:
import nordstromDescStats as nds #see nordstromDescStats.py
import pandas as pd

from bokeh.charts import Bar, show, output_notebook
from bokeh.plotting import figure, show, ColumnDataSource, Legend
from bokeh.models import HoverTool
output_notebook()

In [ ]:
#To process the data into these CSVs I used the following query in the 'nordstrom.db' SQLite database:
#    
#    .open nordstrom.db
#    .mode csv
#    .output <whichever dataset>.csv
#    SELECT * FROM <whichever table> WHERE Color != '' AND Price != 0.00 SORT BY Color;
#
#...and we've got two datasets to play with.
#More processing was done via Perl one liners to remove any lingering strange characters or lines.
menCsv = pd.DataFrame.from_csv('nM.csv')
womenCsv = pd.DataFrame.from_csv('nW.csv')

In [ ]:
#just looking at some of the metadata
menStats = nds.descriptiveStats(menCsv)
womenStats = nds.descriptiveStats(womenCsv)

In [ ]:
#some of this plotting implementation can be hidden so things don't look so messy
mdBar = Bar(
            {'men':menStats.summary.values(),'women':womenStats.summary.values()},
            cat=menStats.summary.keys(),
            legend=True,title="Number of Terms in Each Category",
            width=900,height=400,
            xlabel='Term Groups',ylabel="Number of Terms Counted",
            tools=None
            )
show(mdBar)
    

###Figure 1. Number of Total Items Counted
In terms of fits and types of items there were more total terms for men; the same is true for textures and patterns for women.

In [ ]:
mduBar = Bar(
            {'men':menStats.uniqueSummary.values(),'women':womenStats.uniqueSummary.values()},
            cat=menStats.uniqueSummary.keys(),
            legend=True,title="Number of Unique Terms in Each Category",
            width=900,height=400,
            xlabel='Term Groups',ylabel="Number of Unique Terms",
            tools=None
            )
show(mduBar)

###Figure 2. Number of Total Items Counted
In terms of the numbers of unique items terms there are consistently more for women than for men. This pattern is especially pronounced in the type of item. 

In [ ]:
#this code can be packaged more neatly
def ratioPrep():
    menSummNorm = [float(i)/sum(menStats.summary.values()) for i in menStats.summary.values()]
    menuSummNorm = [float(i)/sum(menStats.uniqueSummary.values()) for i in menStats.uniqueSummary.values()]
    womenSummNorm = [float(i)/sum(womenStats.summary.values()) for i in womenStats.summary.values()]
    womenuSummNorm = [float(i)/sum(womenStats.uniqueSummary.values()) for i in womenStats.uniqueSummary.values()]

    Summ = menSummNorm + womenSummNorm
    uSumm = menuSummNorm + womenuSummNorm
    desc=[
        'men Fit', 'men Texture', 'men Patterns', 'men Items',
        'women Fit', 'women Texture', 'women Patterns', 'women Items'
        ]
    ratio=[]
    
    for i in range(8):
        ratio.append(uSumm[i]/Summ[i])
    
    return({'x':Summ,'y':uSumm,'desc':desc,'ratio':ratio})


def ratioPlot():
    hover = HoverTool(
        tooltips = [
            ("ratio","@ratio"),
            ("", "@desc"),
        ]
    )    
    
    ratioPlot = figure(plot_width=400, plot_height=400,tools=[hover])
    ratioPlot.line([0,float(0.6)],[0,float(0.6)],line_color='#000000')
    ratioSource = ColumnDataSource(
        data=dict(
            x=ratioPrep()['x'],
            y=ratioPrep()['y'],
            desc=ratioPrep()['desc'],
            ratio=ratioPrep()['ratio']
        )
    )

    ratioPlot.scatter(
                    'x', 'y',
                    size=10,
                    fill_color=[
                                '#F4C2C2','#FF0000','#D73B3E','#800000',
                                '#98FB98','#03C03C','#008000','#006400'
                               ], 
                    source=ratioSource,line_color=None,tools=[hover]
                    )

    return (ratioPlot)

show(ratioPlot())

###Figure 3 Ratio of normalized proportions
#####Hover over points for more detail


In [ ]:
#visualization of data at greater detail
menPlot = nds.plotPrep(menCsv)
womenPlot = nds.plotPrep(womenCsv)

In [ ]:
itemBar = Bar(
            {'men':menPlot.itemZip.values(),'women':womenPlot.itemZip.values()},
            cat=menPlot.itemZip.keys(),
            legend=True,title="Item Counts",
            width=900,height=400,
            xlabel='Items',ylabel="Count",
            tools=None
            )
show(itemBar)

###Figure 4
Counts of all items, organized by gender

In [ ]:
patternBar = Bar(
                 {'men':menPlot.patternZip.values(),'women':womenPlot.patternZip.values()},
                 cat=menPlot.patternZip.keys(),
                 legend=True,title="Pattern Counts",
                 width=900,height=400,
                 xlabel='Patterns',ylabel="Count",
                 tools=None
                )
show(patternBar)

###Figure 5
Counts of all patterns, organized by gender

In [ ]:
fitBar = Bar(
            {'men':menPlot.fitZip.values(),'women':womenPlot.fitZip.values()},
            cat=menPlot.fitZip.keys(),
            legend=True,title="Fit Counts",
            width=900,height=400,
            xlabel='Fits',ylabel="Count",
            tools=None
            )
show(fitBar)

###Figure 6
Counts of all fits, organized by gender

In [ ]:
textureBar = Bar(
                {'men':menPlot.textureZip.values(),'women':womenPlot.textureZip.values()},
                cat=menPlot.textureZip.keys(),
                legend=True,title="Textures Counts",
                width=900,height=400,
                xlabel='Textures',ylabel="Count",
                tools=None
                )
show(textureBar)

###Figure 7
Counts of all textures, organized by gender

In [ ]:
#now to to work on the colors
menColors = nds.colorReduce(menCsv).pdDF
womenColors = nds.colorReduce(womenCsv).pdDF
#Plots of color extraction are coming but for now enjoy the extracted color data.
#I certainly am!!!

In [ ]:
#plots of color extraction to come
testM = nds.colorDataPrep(menColors)
testW = nds.colorDataPrep(womenColors)

In [ ]:
colorTestBar = Bar({'men':testM.freqTable.values(),'women':testW.freqTable.values()},
                    cat=testM.freqTable.keys(),legend=True,tools=None)

show(colorTestBar)

###Figure 8
#####Counts of all binned colors by gender

In [1]:
import nordstromDescStats as nds #see nordstromDescStats.py
import pandas as pd

In [3]:
menCsv = pd.DataFrame.from_csv('nM.csv')
nds.itemReduce(menCsv).df

2343


,ItemName,VendorName,Color,Price,item,pattern,texture,fit
1,Versa Lo Tides Board Shorts,Billabong,Acid,59.95,Shorts,unsorted,unsorted,unsorted
2,Cyclone Wind and Water Repellent Jacket,The North Face,Acid Yellow,60.00,Jacket,unsorted,unsorted,unsorted
3,Canyonlands Full Zip Hoodie,The North Face,Adder Green Heather,85.00,Hoodie,unsorted,unsorted,unsorted
4,Long Sleeve Pima Cotton T Shirt,Lacoste,Admiral Blue,59.50,Shirt,unsorted,Cotton,unsorted
5,Miles Denim Button Front Vest,Jeremiah,Admiral Blue,98.00,Vest,unsorted,unsorted,unsorted
6,Evans Relaxed Fit Jeans,Citizens of Humanity,Advantage Wash,209.00,Jeans,unsorted,unsorted,Relaxed
7,Crewneck Jersey T Shirt,James Perse,Aegean,50.00,Shirt,unsorted,unsorted,unsorted
8,Jersey V Neck T Shirt,James Perse,Aegean,50.00,Shirt,unsorted,unsorted,unsorted
9,Trim Fit Sueded Cotton Jersey Polo,James Perse,Aegean,95.00,Polo,unsorted,Cotton,Trim
10,Graduate Tailored Straight Leg Jeans,AG,Aero,188.00,Jeans,unsorted,unsorted,Straight


In [ ]:
len(menCsv.Price.tolist())